This is one of the deliverables produced from this project: https://library.hkust.edu.hk/ds/project/p002/

> This notebook is created by ZHANG Ka Ho Eric (Year 2 student, BEng in Electronic Engineering, HKUST)


In [1]:
import cv2, requests
import pandas as pd
import urllib.request
import numpy as np
import os

Find accuracy

In [2]:
df_person = pd.read_csv('./newdb/tagPerson.csv')
tagged_person_list = df_person.name[0], df_person.name[50], df_person.name[88]
tagged_person_id_list = [0, 50, 88]
# Chia-wei Woo (吳家瑋), Nancy IP (葉玉如), Paul CHU (朱經武)

df_tag = pd.read_csv('./newdb/tag.csv')
result_df = df_tag[df_tag['personID'] == 0].dropna() # check personID == 0
# for i in tagged_person_id_list[1:]:
#     df = df_tag[df_tag['personID'] == i].dropna()
    
#     result_df.add(df)
#     df.reset_index(inplace=True)
#     print(df.shape[0])
# TO-do: add all tagged person into result_df
result_df.reset_index(inplace=True)

df_photo = pd.read_csv('./newdb/photo.csv')
df_photo['image'] = df_photo["imagepath"].map(lambda x: x.split('/')[2].split('.')[0])

In [3]:
# Download images from website to:
# 1. Save images to 'raw'
# 2. crop and save images by the rectangle tagged boxes from database to 'test/unknown/rectangle'


photo_ID_list = []
for i in range(len(result_df)): #df_list[0] for Chia-wei Woo (吳家瑋)
    id = result_df.loc[i, "photoID"]
    aid = result_df.loc[i, "albumID"]
    c1, c2, c3, c4 = (int(i) for i in result_df.iloc[i, -4:])
    temp = df_photo[(df_photo['image'] == id) & (df_photo['aID'] == aid)]
    if not temp.empty:
        url = "https://digitalimages.hkust.edu.hk/gallery/"+temp['imagepath'].values[0].replace(' ', '%20')
        with urllib.request.urlopen(url) as url:
            s = url.read()
        arr = np.asarray(bytearray(s), dtype=np.uint8)
        img = cv2.imdecode(arr, -1) # 'Load it as it is'
        cv2.imwrite('test/raw/'  + id + '.jpg', img)

        c1 = int(c1 * (img.shape[1]/640))
        c2 = int(c2 * (img.shape[1]/640))
        c3 = int(c3 * (img.shape[1]/640))
        c4 = int(c4 * (img.shape[1]/640))
        cv2.rectangle(img, (c1, c2), (c1+c3, c2+c4), (0, 0, 255), 2) 
        face = img[c2:c2 + c4, c1:c1 + c3]
        cv2.imwrite('test/unknown/rectangle/'  + id + "_cropped.jpg", face)
        photo_ID_list.append([id +'.jpg'])


In [16]:
# Use face-crop-plus to crop images and save in in 'test/unknwon/face-crop-plus'


from face_crop_plus import Cropper
from torch.cuda import is_available


current_directory = os.getcwd()
INPUT_DIR = os.path.join(current_directory, "test/raw")
OUTPUT_DIR = os.path.join(current_directory, "test/unknown/face-crop-plus")

# Set all to False if running on CPU (unless you can wait for a bit)
TEST_QUALITY_ENHANCEMENT = False
TEST_ATTR_GROUPING = False
TEST_MASK_GROUPING = False

if __name__ == "__main__":
    # Initialize as None
    enh_threshold = None
    attr_groups = None
    mask_groups = None

    if TEST_QUALITY_ENHANCEMENT:
        enh_threshold = 0.001

    if TEST_ATTR_GROUPING:
        attr_groups = {"faces_features": [2, 3, 4, 5, 7, 8, 10, 11]}
    
    if TEST_MASK_GROUPING:
        mask_groups = {"hat and eyeglasses": [6, 18], "no_accessories": [-9, -15, -18]}

    # Initialize cropper
    cropper = Cropper(
        output_size=(256, 256),
        output_format="jpg",
        face_factor=0.725,
        strategy="all",
        device = "cuda:0" if is_available() else "cpu",
        enh_threshold=enh_threshold,
        attr_groups=attr_groups,
        mask_groups=mask_groups,
    )


    # Process images in the input dir and save face images to output dir
    cropper.process_dir(input_dir=INPUT_DIR, output_dir=OUTPUT_DIR)

Processing: 100%|██████████| 5/5 [01:25<00:00, 17.02s/it]


In [62]:
from deepface import DeepFace
from sklearn.metrics import f1_score

# TO-do: select img_path in unknown / raw to iterately check all people in knwon

# Note: deepface would automatically detect one face in an image

# --> find out which face would be detected --> take x y width height corrdinates

# Output: save and cropped the detected images in output
# 1: the unknown image was considered as the corresponding known sample image after the process of deepface model
# 0: the unknown image was not considered as the corresponding known sample image after the process of deepface model 
# OR
# 0: the unknown image was fail to process the deepface model because of imcompleted face features such as side-face and masked face

result = "try"
model_name = 'Facenet'
known_path = "known"
unknown_path = "test/unknown/face-crop-plus"
known_images = os.listdir("known")


output = []
for unknown_img in os.listdir(unknown_path):
    img2_path = unknown_path + '/' + unknown_img
    member_check = []
    for known_img in os.listdir(known_path):
        img1_path = known_path + '/' + known_img
        try:
            result = DeepFace.verify(img1_path = img1_path, img2_path = img2_path, model_name = model_name)
            member_check.append(int(result["verified"]))
        except:
            member_check.append(0)
    y_pred = np.array(member_check)
    member_check.insert(0, unknown_img)
    if("-NG" in unknown_img):
        y_true = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0])  
    else:
        y_true = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    member_check.append(f1_score(y_true, y_pred, average = 'macro'))
    output.append(member_check)
    print(member_check)

known_images.insert(0, "unknwon_image")
known_images.append("F1-score")


In [31]:
# output, photo_ID_list, result_df, df_photo, df_person, df_tag
len(output[0]) * len(output)

9042

In [69]:
# trun output into csv file

output.pop(0)
ans = pd.DataFrame(output)
ans.set_axis(known_images, axis="columns")

ans.to_csv("test-output.csv", index=False, header=known_images)



In [82]:
# finding positive img and negative img

positive_img = []
for unknown_img in output:
    for check_index in range(len(unknown_img)):
        if unknown_img[check_index] == True and check_index >= 23 and check_index <= 25:
            positive_img.append(unknown_img[0])
            img = cv2.imread("test/unknown/face-crop-plus/" + unknown_img[0])
            cv2.imwrite('output/target/'  + unknown_img[0], img)

positive_img

negative_img = os.listdir("test/unknown/face-crop-plus")
print(len(negative_img) - 23 - 20 - 40)

#     P   N
# T   23  20
# F   40  191


191


In [4]:
# Using another model (encoding also) to do face recognition


import face_recognition

def read_img(path):
    img = cv2.imread(path)
    (h, w) = img.shape[:2]
    width = 500
    ratio = width / float(w)
    height = int(h * ratio)
    return cv2.resize(img, (width, height))



acc_list = []

known_encodings = []
known_names = []
known_dir = 'known' # +  '/' + tagged_person
for file in os.listdir(known_dir):
  img = read_img(known_dir + '/' + file)
  img_enc = face_recognition.face_encodings(img)[0]
  known_encodings.append(img_enc)
  known_names.append(file.split('.')[0])

unknown_dir = 'test/unknown/face-crop-plus'
for file in os.listdir(unknown_dir):
  # print("Processing", file)
  img = read_img(unknown_dir + '/' + file)
  img_enc = face_recognition.face_encodings(img)
  if(img_enc == []):
    acc_list.append(0)
    continue

  results = face_recognition.compare_faces(known_encodings, img_enc[0])
  accuracy = np.sum(np.array(results)) / len(results)
  if(accuracy > 0.75): # change
    acc_list.append(1)
  else:
    acc_list.append(0)



In [7]:
tagged_person = tagged_person_list[0][:-6] # @@@replace with@@@ for tagged_person in tagged_person_list: 

record_list = []
for index in range(len(photo_ID_list)):
    file_name = photo_ID_list[index][0]
    cropped_file_name = file_name[:-4] +  '_cropped' + '.jpg'
    correct = acc_list[index]
    if correct:
        detected_person = tagged_person
    else:
        detected_person = 0
    record_list.append([file_name, tagged_person, cropped_file_name, detected_person, correct])

result = pd.DataFrame(record_list)
result.to_csv("face-recognition_output.csv")

Output all image in website with important ust members

In [ ]:
# Download all images from website to:
# 1. Save images to 'raw'
# 2. crop and save images by the rectangle tagged boxes from database to 'unknown/rectangle'

# @@@@@ TO BE IMPLEMENTED @@@@@

result_df = df_tag
for i in range(len(result_df)): #df_list[0] for Chia-wei Woo (吳家瑋)
    id = result_df.loc[i, "photoID"]
    aid = result_df.loc[i, "albumID"]
    temp = df_photo[(df_photo['image'] == id) & (df_photo['aID'] == aid)]
    if not temp.empty:
        url = "https://digitalimages.hkust.edu.hk/gallery/"+temp['imagepath'].values[0].replace(' ', '%20')
        with urllib.request.urlopen(url) as url:
            s = url.read()
        arr = np.asarray(bytearray(s), dtype=np.uint8)
        img = cv2.imdecode(arr, -1) # 'Load it as it is'
        cv2.imwrite('raw/'  + id + '.jpg', img)